In [1]:
from pydantic import BaseModel

In [110]:
class ConsultingResult(BaseModel):
    short_name: str
    name: str
    value: float
    units: str
    display_units: str

In [112]:
from dataclasses import dataclass, field
import json
import os
from pathlib import Path
from typing import Any, Dict, NamedTuple, Tuple
from pprint import pprint
import functools

def requires_write_permission(fun):
    @functools.wraps(fun)
    def wrapper(*args, **kwargs):
        if not os.getenv('DB_WRITEMODE'):
            raise PermissionError("Write permission disabled")
        return fun(*args, **kwargs)
    return wrapper

@dataclass
class ConsultingResultRepo:
    consulting_results: list[ConsultingResult] = field(repr=False)
    path: Path
    write_mode: bool

    @classmethod
    def connect(cls, path, write_mode = False) -> 'ConsultingResultRepo':
        path = Path(path)
        results = []
        for file in path.glob('*.json'):
            result = ConsultingResult.model_validate_json(file.read_text())
            results.append(result)
        
        return ConsultingResultRepo(
            consulting_results=results,
            path=path,
            write_mode = write_mode
        )

    @requires_write_permission
    def clear_all(self) -> None: 
        self.consulting_results.clear()
            
    @requires_write_permission
    def put(self, short_name: str, name: str, value: float, units: str, display_units: str) -> None:
        result = ConsultingResult(
            short_name=short_name,
            name=name,
            value=value,
            units=units,
            display_units=display_units,
        )
        self.consulting_results.append(result)

    def list_all(self) -> list[ConsultingResult]:
        pprint(self.consulting_results)

    def get(self, s_name: str) -> ConsultingResult:
        for result in self.consulting_results:
            if s_name in result.short_name:
                return result
    
    def to_dict(self) -> Dict[Any, Any]:
        return self._asdict()
    
    def save(self):
        path = Path(self.path)
        if path.exists():
            assert path.is_dir()
        
        path.mkdir(parents=True, exist_ok=True)
        for i, result in enumerate(self.consulting_results):
            json_text = result.model_dump_json(indent=3)
            path.joinpath(f'{i}.json').write_text(json_text)
        

In [101]:
new_repo = ConsultingResultRepo.connect(path='data')
new_repo

ConsultingResultRepo(path=WindowsPath('data'), write_mode=False)

In [102]:
new_repo.clear_all()

PermissionError: Write permission disabled

In [105]:
os.environ['DB_WRITEMODE'] = '1'

In [106]:
new_repo.clear_all()

In [54]:
a = result: ConsultingResult(
    short_name='n_sess',
    name='Number of Sessions',
    value=100,
    units='Sessions',
    display_units='Sess'
)

a # value is automatically converted to float from int by pydantic

ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess')

In [55]:
b = result: ConsultingResult(
    short_name='tot_hours',
    name='Total Number of Hours',
    value=10.5,
    units='Hour',
    display_units='Hrs'
)

b

ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs')

In [59]:
results = ConsultingResultRepo(
    consulting_results=[a, b],
    path='data/'
)
results
# Tuple automatically converted to List

ConsultingResultRepo(consulting_results=[ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'), ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs')], path='data/')

In [60]:
results.save()

In [57]:
# let's add a new consulting result to the repo using the put method
c = result: ConsultingResult(
    short_name='n_sc',
    name='Number of Short Chats',
    value=50,
    units='Session',
    display_units='Sess'
)

results.put(c)
results

ConsultingResultRepo(consulting_results=[ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'), ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs'), ConsultingResult(short_name='n_sc', name='Number of Short Chats', value=50.0, units='Session', display_units='Sess')], path='.')

In [48]:
# display all results using list_all method
results.list_all()

[ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'),
 ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs'),
 ConsultingResult(short_name='n_sc', name='Number of Short Chats', value=50.0, units='Session', display_units='Sess')]


In [49]:
# get a result by specifying the short name (e.g., n_sess)
results.get('n_sess')

ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess')

In [50]:
# get a dictionary of the table
results.to_dict()

{'consulting_results': [ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'),
  ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs'),
  ConsultingResult(short_name='n_sc', name='Number of Short Chats', value=50.0, units='Session', display_units='Sess')],
 'path': '.'}

In [51]:
# testing conversion to pandas dataframe

import pandas as pd
pd.DataFrame(results.to_dict()['consulting_results'])

,0,1,2,3,4
0,"(short_name, n_sess)","(name, Number of Sessions)","(value, 100.0)","(units, Sessions)","(display_units, Sess)"
1,"(short_name, tot_hours)","(name, Total Number of Hours)","(value, 10.5)","(units, Hour)","(display_units, Hrs)"
2,"(short_name, n_sc)","(name, Number of Short Chats)","(value, 50.0)","(units, Session)","(display_units, Sess)"


In [12]:
# delete results using clear_all 
print('Before clear all')
pprint(results.consulting_results)
print()
print('After clear all')
results.clear_all()
results.consulting_results

Before clear all
[ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'),
 ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs'),
 ConsultingResult(short_name='n_sc', name='Number of Short Chats', value=50.0, units='Session', display_units='Sess')]

After clear all


[]

In [16]:
ConsultingResultRepo.put(result: ConsultingResult(short_name="BestRes", name="ResultOfAll", value=100, units="None needed.", display_units="!!!"))

TypeError: ConsultingResultRepo.put() missing 1 required positional argument: 'ConsultingResult'

In [17]:
r = ConsultingResultRepo(
    consulting_results=[a, b, c]
)

In [18]:
r.list_all()

[ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'),
 ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs'),
 ConsultingResult(short_name='n_sc', name='Number of Short Chats', value=50.0, units='Session', display_units='Sess')]


In [58]:
results.save()

In [108]:
new_results = ConsultingResultRepo.connect(path='data')
new_results.list_all()

[ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess'),
 ConsultingResult(short_name='tot_hours', name='Total Number of Hours', value=10.5, units='Hour', display_units='Hrs')]


In [109]:
new_results.get('n_sess')

ConsultingResult(short_name='n_sess', name='Number of Sessions', value=100.0, units='Sessions', display_units='Sess')